In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.dtypes

In [ ]:
dataset.info()
dataset.isna().sum()

In [ ]:
labels = dataset.pop('expenses')
labels

In [ ]:
# Replace categorical values to numerical
# first check the unique values

# sex column
print(dataset.sex.unique())  # ['female' 'male']
dataset["sex"].replace(to_replace=['female', 'male'], value = [0,1], inplace=True)

# smoker column
print(dataset.smoker.unique())  # ['yes' 'no']
dataset["smoker"].replace(to_replace=['yes', 'no'], value = [0,1], inplace=True)

# region column
print(dataset.region.unique())  # ['southwest' 'southeast' 'northwest' 'northeast']
dataset["region"].replace(to_replace=['southwest', 'southeast', 'northwest', 'northeast'],
                          value = [0,1,2,3], inplace=True)
dataset

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

In [ ]:
# model
# normalizer = tf.keras.layers.Normalization(axis=-1)

model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(6,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.MeanSquaredError()])

In [ ]:
%%time
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2,
    # Early stopping prevent overfitting, if the model is not getting any better
    callbacks=[keras.callbacks.EarlyStopping(patience=5)])

In [ ]:
history.history.keys()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
